# **Elgamal Public Key Encryption** [Prepared By Jevin Evans and Muhammad Ismail]
---

The **Elgamal Public Key Encryption** is a public-key cryptosystem based on the [Diffie-Hellman](https://en.wikipedia.org/wiki/Diffie%E2%80%93Hellman_key_exchange) key exchange protocol. Elgamal uses modular exponentiation to encrypt and decrypt data for secure information exchange.

The following will walk through an example of the algorithm and the inner workings, including global parameter use, key generation, encryption, and decryption.

## **Instructions** 
Run in order by hitting the play button next to each code box or in the <u>menu bar</u>, select **Runtime->Run All** to run the whole lab.

In [ ]:
# @title Plaintext Message
# @markdown Insert a message that you want to encrypt in the demonstration or use the example provided.

PLAINTEXT = "This is a message that will be used as an example to demonstrate the Elgamal Public Key Encryption." # @param {type:"string"}

from random import randint
from math import pow, sqrt

# Global Variables
---

In **Elgamal**, the use of global parameters and prime numbers is essential. Anyone with a key will have the **Global Prime** and **Global Primitive Root** values used for their system. When users are on the same system, these numbers are the same, and when on different systems, they will be different.

## Prime Number

Operations in Elgamal are done over the modulo of the **Global Prime** number. This prime number is represented as
$ P $. In most cryptosystems, the use of large, random prime numbers effectively avoids cryptoanalysis and eavesdropping on communications. In this example, smaller values will be used for convenience.


To get the **Global Primitive**, a random number will be selected from $[10^2..10^4]$

In [ ]:
# Checks if a number is prime
def isPrime(p:int):
  if p <= 1: return False
  if p == 2: return True

  for x in range(2, int(sqrt(p))+1):
    if p % x == 0: return False
  return True


# Choose Random Prime
PRIME = randint(pow(10,2), pow(10,4))
while not isPrime(PRIME):
  PRIME = randint(pow(10,2), pow(10,4))

print(f"Global Prime Number: {PRIME}")

Global Prime Number: 1993


## Primitive Root
Definition ([Wikipedia](https://en.wikipedia.org/wiki/Primitive_root_modulo_n)): "A number **g** is a primitive root modulo **n** if every number **a** coprime to **n** is congruent to a power of **g** modulo **n**. That is, **g** is a primitive root modulo n, if for every integer a coprime to n, there is some integer k for which $g^k ≡ $ $a$ $ (mod$ $n)$."

For example, if the prime number was 7 and we wanted to see if 3 is a primitive root modulo 7, we can validate by testing $3^k ≡$ $a$ $(mod$ $n)$ and if all the modulo results cover [1..6], then 3 is a primitive root of 7.

$g^k$ | Result | Modulo
---|---|---
$3^1$ | 3    | 3 (mod 7) = 3
$3^2$ | 9    | 9 (mod 7) = 2
$3^3$ | 27   | 27 (mod 7) = 6
$3^4$ | 81   | 81 (mod 7) = 4
$3^5$ | 243  | 243 (mod 7) = 5
$3^6$ | 729  | 729 (mod 7) = 1
$3^7$ | 2187 | 2187 (mod 7) = 3

You can see all values from 1 to 6; therefore, 3 is a primitive root of 7.

---

The **Global Primitive** is represented as $g$. 

The following algorithm is an efficient way to find all of the primitive roots given a prime number using Eulers Totient and then returns a random primitive root from the list. 
[Primitive Root Algorithm Explained](https://cp-algorithms.com/algebra/primitive-root.html).



In [ ]:
# Computes the modular exponentiation of a number
# Helps to find primitive roots/coprime numbers
def powmod(a, b, p):
  res = 1
  while b:
      if b & 1:
          res = int(res * 1 * a % p)
          b -= 1
      else:
          a = int(a * a % p)
          b >>= 1
  return res

# Finds all primitive roots and returns a random root
def findPrimitiveRoot(p):
  fact = list()
  phi = p - 1
  n = phi

  for i in range(2, n+1):
      if n % i == 0:
          fact.append(i)
          while n % i == 0:
              n /= i
  if n > 1:
      fact.append(n)

  primitives = list()
  for i in range(2, p):
      ok = True
      for x in range(len(fact)):
          ok &= powmod(i, int(phi/fact[x]), p) != 1
      if ok:
          primitives.append(i)

  # Returns a random primitive root
  if len(primitives) > 0:
      return primitives[randint(0, len(primitives)-1)]
  return -1


PRIMITIVE = findPrimitiveRoot(PRIME)
print(f'Global Primitive Root: {PRIMITIVE}')

Global Primitive Root: 1855


We have now established the global values that users on a single system will use to communicate with each other. If users on different systems want to communicate, they will have different **Global Prime** and **Global Primitive Root**.

# Key Generation
---

Like all Public Key Cryptographic systems, there are private keys and public keys. The private keys are kept secret to the user, and the public keys are shared. The following will explain how these keys are generated and shared in Elgamal.

## Private Key
The private key is chosen randomly from the list of values from $[2..P-2]$.

Represented as $X = $ random$([2..P-2])$

If Alice and Bob were on the same system and needed to get new keys, their private keys would be.


In [ ]:
# Generate Alice's Private Key
alicePrivateKey = randint(2, PRIME-2)

# Generate Bob's Private Key
bobPrivateKey = randint(2, PRIME-2)

print(f"Alice's Private Key: {alicePrivateKey}")
print(f"Bob's Private Key: {bobPrivateKey}")

Alice's Private Key: 676
Bob's Private Key: 1495


## Public Key
The public key is generated using modular exponentiation using the private key, the **Global Prime**(P), and the **Global Primitive Root**(g).

Represented as $ Y = g^{X} $mod $ P$ 

where:
- Y is the public key
- g is the primitive root
- X is the private key
- P is the prime

Now we can use Alice and Bob's public keys.

In [ ]:

# Alice's Public Key
alicePublicKey = PRIMITIVE**alicePrivateKey % PRIME

print(f"Alice's Key Pair: \n\tPrivate: {alicePrivateKey}\n\tPublic: {alicePublicKey}\n")

# Bob's Public Key
bobPublicKey = PRIMITIVE**bobPrivateKey % PRIME

print(f"Bob's Key Pair: \n\tPrivate: {bobPrivateKey}\n\tPublic: {bobPublicKey}")

Alice's Key Pair: 
	Private: 676
	Public: 335

Bob's Key Pair: 
	Private: 1495
	Public: 502


Once a user has their public key value, they will publish their full public key as a tuple of their **Global Prime**(P), **Global Primitive**(g), and their specific **Public Key**(Y) for other users to use to send them messages.  

Published Public Key format:  (P, g, Y)

Now Alice and Bob can publish their public keys.

In [ ]:
print("Alice's Public Key Tuple:", (PRIME, PRIMITIVE, alicePublicKey))
print("Bob's Public Key Tuple:", (PRIME, PRIMITIVE, bobPublicKey))

Alice's Public Key Tuple: (1993, 1855, 335)
Bob's Public Key Tuple: (1993, 1855, 502)


# Encryption
---

When someone wants to send an encrypted message in Elgamal, they need to generate 2 values $C_1$ and $C_2$. These values are sent as a tuple making up the encrypted message.


## $C_1$
$C_1$ is used for verification and the decryption of $C_2$. It is generated using a **k** value and the **Global Primitive** and **Global Prime**. 

**k** is a random number from $[2..P-2]$, and **k** is coprime with **P-1**. Meaning the greatest common divisor between the **k** and **P-1** is 1. Then by raising the **Global Primitive** to the **k** value modulo the **Global Prime**, $C_1$ can be computed.

$k = \{2...P-2\}$

$C_1 = g^k$ mod $P$

In [ ]:
# Returns the Greatest Common Divisor between 2 numbers
def gcd(a, b):
  if a == 0:
    return b
  return gcd(b%a, a)

# Generate k
k = randint(2, PRIME - 2)

# Verifies that k is coprime to 
while gcd(k, PRIME - 1) != 1:
  k = randint(2, PRIME - 2)

print(f"Random k-value = {k}")

# Compute C1
C1 = PRIMITIVE**k % PRIME

print(f"C1 = {C1}")


Random k-value = 79
C1 = 91


## $C_2$
$C_2$ holds the encrypted message. The message is encrypted using a computed secret **shared key** using **k** and the receiver's **Public Key**(Y). The secret **shared key** (s) is generated by raising the receiver's public key to the power of **k** modulo the **Global Prime**. The shared key is then multiplied by the message to complete the encryption.

$s = Y^k $ mod $P$

$C_2 = m*s$ mod $P$

where $Y$ is the recipient's public key, and $m$ is the message being encrypted.

If Bob wanted to send an encrypted message to Alice, he would use her **public key** when computing the **shared key**.

In [ ]:
# Generate shared key
sharedKey = alicePublicKey**k % PRIME

print(f"Shared Key = {sharedKey}")

# Compute/Encrypt Messages C2
# Convert message to decimal
print(f"\nPlaintext:", PLAINTEXT)
print(f"Plaintext (Decimal):", [ord(i) for i in PLAINTEXT])

print(f"\n\tExample: C2 = {sharedKey}*{ord(PLAINTEXT[0])} mod {PRIME} = {sharedKey*ord(PLAINTEXT[0]) % PRIME}")

C2 = [(sharedKey*ord(i)) % PRIME for i in PLAINTEXT]

print(f'\nChiphertext (Decimal): {C2}')
C2 = ''.join([chr(i) for i in C2])
print(f'Chiphertext: {C2}')

Shared Key = 242

Plaintext: This is a message that will be used as an example to demonstrate the Elgamal Public Key Encryption.
Plaintext (Decimal): [84, 104, 105, 115, 32, 105, 115, 32, 97, 32, 109, 101, 115, 115, 97, 103, 101, 32, 116, 104, 97, 116, 32, 119, 105, 108, 108, 32, 98, 101, 32, 117, 115, 101, 100, 32, 97, 115, 32, 97, 110, 32, 101, 120, 97, 109, 112, 108, 101, 32, 116, 111, 32, 100, 101, 109, 111, 110, 115, 116, 114, 97, 116, 101, 32, 116, 104, 101, 32, 69, 108, 103, 97, 109, 97, 108, 32, 80, 117, 98, 108, 105, 99, 32, 75, 101, 121, 32, 69, 110, 99, 114, 121, 112, 116, 105, 111, 110, 46]

	Example: C2 = 242*84 mod 1993 = 398

Chiphertext (Decimal): [398, 1252, 1494, 1921, 1765, 1494, 1921, 1765, 1551, 1765, 469, 526, 1921, 1921, 1551, 1010, 526, 1765, 170, 1252, 1551, 170, 1765, 896, 1494, 227, 227, 1765, 1793, 526, 1765, 412, 1921, 526, 284, 1765, 1551, 1921, 1765, 1551, 711, 1765, 526, 1138, 1551, 469, 1195, 227, 526, 1765, 170, 953, 1765, 284, 526, 469, 953, 711, 1921

Bob will then send Alice the encrypted ciphertext tuple (C1, C2).

In [ ]:
print((C1, C2))

(91, 'ƎӤזށۥזށۥ؏ۥǕȎށށ؏ϲȎۥªӤ؏ªۥ\u0380זããۥ܁ȎۥƜށȎĜۥ؏ށۥ؏ˇۥȎѲ؏ǕҫãȎۥªιۥĜȎǕιˇށªڏ؏ªȎۥªӤȎۥ˲ãϲ؏Ǖ؏ãۥ֏Ɯ܁ãז*ۥÕȎդۥ˲ˇ*ڏդҫªזιˇҏ')


# Decryption
---

Once Alice receives a message, she will use her private key to decrypt the message. To do this, she will need to recompute the **shared key**, which is the result of raising $C_1$ to her **private key** modulo the **Global Prime**. Once the **shared key** is recomputed, she multiplies the **ciphertext** by the **shared key** raised to the **Global Prime** - 2. The math theory behind the Elgamal scheme is why this shared key can be recomputed without being shared for decryption.

$ sharedKey = C_1^{X} $ mod $P$

$ Plaintext = C_2 * sharedKey^{P - 2} $ mod $P$

In [ ]:
print(f"Encrypted Message Received (C1, C2): {(C1, C2)}")

# Generate reverse shared key
sharedKey = C1**alicePrivateKey % PRIME
print(f"\nRecomputed Shared Key: {sharedKey}")

print(f"\nCiphertext: {C2}")
print(f"Ciphertext (Decimal): {[ord(i) for i in C2]}")

print(f"\n\tExample:  M = {ord(C2[0])}*{sharedKey}^{PRIME-2} mod {PRIME} = {ord(C2[0])*sharedKey**(PRIME-2) % PRIME}")
decrypted = [ord(x)*sharedKey**(PRIME-2) % PRIME for x in C2]

print(f"\nDecrypted (Decimal): {decrypted}")
print(f"Decrypted: {''.join([chr(i) for i in decrypted])}")

Encrypted Message Received (C1, C2): (91, 'ƎӤזށۥזށۥ؏ۥǕȎށށ؏ϲȎۥªӤ؏ªۥ\u0380זããۥ܁ȎۥƜށȎĜۥ؏ށۥ؏ˇۥȎѲ؏ǕҫãȎۥªιۥĜȎǕιˇށªڏ؏ªȎۥªӤȎۥ˲ãϲ؏Ǖ؏ãۥ֏Ɯ܁ãז*ۥÕȎդۥ˲ˇ*ڏդҫªזιˇҏ')

Recomputed Shared Key: 242

Ciphertext: ƎӤזށۥזށۥ؏ۥǕȎށށ؏ϲȎۥªӤ؏ªۥ΀זããۥ܁ȎۥƜށȎĜۥ؏ށۥ؏ˇۥȎѲ؏ǕҫãȎۥªιۥĜȎǕιˇށªڏ؏ªȎۥªӤȎۥ˲ãϲ؏Ǖ؏ãۥ֏Ɯ܁ãז*ۥÕȎդۥ˲ˇ*ڏդҫªזιˇҏ
Ciphertext (Decimal): [398, 1252, 1494, 1921, 1765, 1494, 1921, 1765, 1551, 1765, 469, 526, 1921, 1921, 1551, 1010, 526, 1765, 170, 1252, 1551, 170, 1765, 896, 1494, 227, 227, 1765, 1793, 526, 1765, 412, 1921, 526, 284, 1765, 1551, 1921, 1765, 1551, 711, 1765, 526, 1138, 1551, 469, 1195, 227, 526, 1765, 170, 953, 1765, 284, 526, 469, 953, 711, 1921, 170, 1679, 1551, 170, 526, 1765, 170, 1252, 526, 1765, 754, 227, 1010, 1551, 469, 1551, 227, 1765, 1423, 412, 1793, 227, 1494, 42, 1765, 213, 526, 1380, 1765, 754, 711, 42, 1679, 1380, 1195, 170, 1494, 953, 711, 1167]

	Example:  M = 398*242^1991 mod 1993 = 84

Decrypted (Decimal): [84, 104, 105, 115, 32, 105, 115, 32, 97, 32, 109, 101, 115, 115, 97, 103,

# Conclusion
---

Now you have a better understanding of how key generation, encryption, and decryption work in Elgamal Public Key Cryptography.


# Resources
1. https://en.wikipedia.org/wiki/ElGamal_encryption 
1. https://www.geeksforgeeks.org/elgamal-encryption-algorithm/ 
1. https://www.commonlounge.com/discussion/35a1c2baa00b447f9275e8f71b02ef29#elgamal-signature-scheme 
1. https://en.wikipedia.org/wiki/ElGamal_signature_scheme
1. https://www.commonlounge.com/discussion/26ee8f79aaa74d758f3a9429e28c8c6c 
1. https://primes.utm.edu/glossary/page.php?sort=RelativelyPrime#:~:text=Two%20integers%20are%20relatively%20prime,12%20and%2014%20are%20not. 
